In [1]:
import os
os.chdir("/mnt/c/Users/Dave/Project/COBS")
import sys
import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from cobs import Model, Reward
from cobs import OccupancyGenerator as OG
from tqdm import tqdm
from pprint import pprint

from config import state_names, disturbances_dict, eplus_naming_dict, eplus_var_types
from DQN import *

In [2]:
Model.set_energyplus_folder("/usr/local/EnergyPlus-9-3-0/")
idf_files_path = "/mnt/c/users/Dave/Downloads/idf-sample-files/"

# idf_path = idf_files_path+"2020/RefBldgLargeOfficeNew2004_Chicago.idf"
idf_path = idf_files_path+"2020/RefBldgMediumOfficeNew2004_Chicago.idf"
epw_path = "cobs/data/weathers/USA_IL_Chicago-OHare.Intl.AP.725300_TMY3.epw"

ep_model = setup_env(idf_path, epw_path, num_days=14, timestep=4)

In [3]:
ep_model.simulate()

EnergyPlus Starting
EnergyPlus, Version 9.3.0-baff08990c, YMD=2022.03.24 17:57
Initializing Response Factors
Calculating CTFs for "STEEL FRAME NON-RES EXT WALL", Construction # 1
Calculating CTFs for "IEAD NON-RES ROOF", Construction # 2
Calculating CTFs for "EXT-SLAB", Construction # 3
Calculating CTFs for "INT-WALLS", Construction # 4
Calculating CTFs for "INT-FLOOR-TOPSIDE", Construction # 5
Calculating CTFs for "DROPCEILING", Construction # 8
Calculating CTFs for "INTERIORFURNISHINGS", Construction # 9
Initializing Window Optical Properties
Initializing Solar Calculations
Allocate Solar Module Arrays
Initializing Zone and Enclosure Report Variables
Initializing Surface (Shading) Report Variables
Determining Shadowing Combinations
Computing Window Shade Absorption Factors
Proceeding with Initializing Solar Calculations
Initializing Surfaces
Initializing Outdoor environment for Surfaces
Setting up Surface Reporting Variables
Initializing Temperature and Flux Histories
Initializing Wi

EnergyPlus Completed Successfully.


In [4]:
df = pd.DataFrame(ep_model.state_history)
# Replace year to 1991
df['time'] = df['time'].mask(df['time'].dt.year > 1, df['time'] + pd.offsets.DateOffset(year=1991))
df.index = df['time']
df.drop(columns=['time'], inplace=True)
df.head()

/home/dave/.local/lib/python3.8/site-packages/pandas/core/arrays/datetimes.py:760: PerformanceWarning: Non-vectorized DateOffset being applied to Series or DatetimeIndex.
  warnings.warn(


,timestep,temperature,occupancy,terminate,reward,PMV,PPD,Lights,Fans Cost,Cooling Cost,...,System Node Setpoint Temperature_VAV_2 SUPPLY EQUIPMENT OUTLET NODE,System Node Setpoint Temperature_VAV_3 SUPPLY EQUIPMENT OUTLET NODE,AHU1 MA Temp.,AHU2 MA Temp.,AHU3 MA Temp.,Wind Speed,Wind Direction,Diffuse Solar Rad.,Direct Solar Rad.,HVAC Power
time,,,,,,,,,,,,,,,,,,,,,
1991-07-01 00:15:00,0,"{'Core_bottom': 24.637464688560218, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,NaN,"{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.622754,25.311911,22.989326,3.250,347.5,0.0,0.0,8.440094
1991-07-01 00:30:00,1,"{'Core_bottom': 24.632549236026296, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23674295502753231, 'Core_mi...","{'Core_bottom': 6.163033888512302, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.615844,25.305343,22.938026,3.400,15.0,0.0,0.0,8.440094
1991-07-01 00:45:00,2,"{'Core_bottom': 24.627573422207853, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23754809813081051, 'Core_mi...","{'Core_bottom': 6.1709768620456344, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.607828,25.297574,22.882158,3.550,42.5,0.0,0.0,8.440094
1991-07-01 01:00:00,3,"{'Core_bottom': 24.622486445356554, 'Core_mid'...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23838046384231054, 'Core_mi...","{'Core_bottom': 6.179217004316627, 'Core_mid':...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.599430,25.289313,22.827864,3.700,70.0,0.0,0.0,8.440094
1991-07-01 01:15:00,4,"{'Core_bottom': 24.61727913492755, 'Core_mid':...","{'Core_bottom': 0.0, 'Core_mid': 0.0, 'Core_to...",False,-983689.186327,"{'Core_bottom': -0.23924888109050627, 'Core_mi...","{'Core_bottom': 6.1878450497201385, 'Core_mid'...","{'Core_bottom': 476228.40807524405, 'Core_mid'...",0.0,0.0,...,12.8,12.8,24.590807,25.280680,22.774796,3.725,75.0,0.0,0.0,8.440094


In [5]:
def get_avg_PPD(df, working_time):
    # Get PPD only when in working hours and zone is occupied
    means = []
    for i in range(len(df)):
        curr = df.iloc[i]
        PPDs = []
        if curr.name.hour in working_time:
            for zone in curr['PPD'].keys():
                if curr['occupancy'][zone] > 0:
                    PPDs.append(curr['PPD'][zone])
        if len(PPDs) == 0:
            mean = 0
        else:
            mean = np.mean(PPDs)
        means.append(mean)
    return np.mean(means)


def getStatics(df, working_time):
    meanPPD = get_avg_PPD(df, working_time)
    print("Average PPD = ", meanPPD)
    
    HVAC = []
    for i in range(len(df)):
        curr = df.iloc[i]
        if curr.name.hour in working_time:
            HVAC.append(curr['HVAC Cost'])
    HVACPower = np.sum(HVAC)
#     HVACPower = np.sum(df["HVAC Power"])
    HVACEnergy = HVACPower * 900 / 1000 / 3600 # Unit in kWh
    print("Energy Consumed by the HVAC System = {:.2f}kWh".format(HVACEnergy))
    
working_time = range(9, 17)  
getStatics(df, working_time)

Average PPD =  2.405406190376174
Energy Consumed by the HVAC System = 4375333.86kWh


In [6]:
results_dir = 'simulation_results'

if not os.path.exists(results_dir):
    os.mkdir(results_dir)
    
df.to_pickle(os.path.join(results_dir, "Sim-chicago.pkl"))
print('Saved!')

Saved!
